# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4310029237#4088494919
date='November 8, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,21.1km/h,78.8km/h
1,Calories,555,555
2,Elapsed Time,4:13:09,4:13:09


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,INDIANA - COLUMBUS SCHOOL 1.08km 46m 4.2%,5:21,12.2km/h,231W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
1,Repecho Columbus School 0.23km 25m 10.6%,1:55,7.3km/h,255W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
2,sprint ultimos 300 alto palmas 0.32km 15m 4.8%,1:33,12.4km/h,224W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
3,Alto - Doña rosa (bajando) 3.25km 269m -8.3%,4:44,41.3km/h,114W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
4,Palmas-Escobero 9.81km 925m -9.4%,14:32,40.5km/h,49W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
5,Bajada Transversal - Intermedia 5.68km 549m ...,8:48,38.8km/h,17W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
6,Doña Rosa - Tubos EPM 2.27km 159m -7.0%,3:12,42.6km/h,9W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
7,Bajada transversal de la montaña 3.31km 248m...,4:26,44.8km/h,31W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
8,City Plaza - Hospital 1.10km 105m -9.5%,2:49,23.5km/h,28W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."
9,Av Poblado (Cll 40 Sur) - Clinica de la Polici...,3:14,19.7km/h,193W,"{'Calories': {'Avg': '555', 'Max': '555'}, 'El..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df.to_json(db_file)

### Save notebook here

In [9]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [10]:
!bash commit.sh

[master 752a663] Morning Ride 6:07 AM on Monday, November 16, 2020
 2 files changed, 3 insertions(+), 19 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 1.67 KiB | 113.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   6e2a669..752a663  master -> master


In [11]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.125)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Linux 4.19.0-10-amd64 x86_64)
